In [ ]:
import pyodbc
import pandas as pd

# SQL Server connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=db_Churn;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Get all tables with schema
cursor.execute("SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
tables = [f"{row[0]}.{row[1]}" for row in cursor.fetchall()]

# Get all views with schema
cursor.execute("SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.VIEWS")
views = [f"{row[0]}.{row[1]}" for row in cursor.fetchall()]

all_objects = tables + views

print("Found:", all_objects)

# Export each table/view into Excel
output_file = "db_Churn_data.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for obj in all_objects:
        try:
            df = pd.read_sql(f"SELECT * FROM {obj}", conn)
            # Excel sheet names cannot exceed 31 characters, replace . with _
            sheet_name = obj.replace(".", "_")[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Exported {obj} ✅")
        except Exception as e:
            print(f"Skipping {obj} ❌ due to error: {e}")

conn.close()

print(f"\n📂 All tables and views exported to {output_file}")
